In [2]:
with open('input.txt',"r", encoding="utf-8") as f:
    text=f.read()

In [3]:
len(text)

1115394

In [4]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [5]:
chars= sorted(list(set(text)))
vocab_size=len(chars)
''.join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [6]:
len(''.join(chars))

65

In [7]:
stoi= {e:i for i, e in enumerate(chars)}
itos=  {i:e for i, e in enumerate(chars)}

encoder=lambda s:[stoi[e] for e in s]
decoder=lambda s:''.join(itos[e] for e in s)

In [8]:
encoder("I am good")

[21, 1, 39, 51, 1, 45, 53, 53, 42]

In [9]:
decoder(encoder("I am good"))

'I am good'

In [10]:
import torch

In [11]:
data=torch.tensor(encoder(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
n=int(len(data)*0.9)
train_data=data[:n]
val_data=data[n:]

In [13]:
n

1003854

In [14]:
block_size=8
train_x=train_data[:block_size]
train_y=train_data[1:block_size+1]

for i in range(block_size):
    x=train_x[:i+1]
    y=train_y[i]
    print("For input {} op is {}".format(x,y))

For input tensor([18]) op is 47
For input tensor([18, 47]) op is 56
For input tensor([18, 47, 56]) op is 57
For input tensor([18, 47, 56, 57]) op is 58
For input tensor([18, 47, 56, 57, 58]) op is 1
For input tensor([18, 47, 56, 57, 58,  1]) op is 15
For input tensor([18, 47, 56, 57, 58,  1, 15]) op is 47
For input tensor([18, 47, 56, 57, 58,  1, 15, 47]) op is 58


In [15]:
batch_size=4
block_size=8

def get_batch(split):
    data=train_data if split=="train" else val_data
    ix=torch.randint(len(data)-block_size, (batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x,y



In [16]:
xb,yb=get_batch('train')

In [17]:
print("inputs",xb)
print("outputs",yb)

inputs tensor([[51, 39, 63,  1, 39, 52, 57, 61],
        [50, 47, 44, 44, 53, 56, 42,  1],
        [52,  1, 58, 46, 43,  1, 46, 53],
        [39, 58, 57,  1, 58, 46, 43,  1]])
outputs tensor([[39, 63,  1, 39, 52, 57, 61, 43],
        [47, 44, 44, 53, 56, 42,  1, 58],
        [ 1, 58, 46, 43,  1, 46, 53, 57],
        [58, 57,  1, 58, 46, 43,  1, 51]])


In [18]:
for i in range(batch_size):
    for j in range(block_size):
        x=xb[i,:j+1]
        y=yb[i,j]
        print("For input {} output is {}".format(x,y))

For input tensor([51]) output is 39
For input tensor([51, 39]) output is 63
For input tensor([51, 39, 63]) output is 1
For input tensor([51, 39, 63,  1]) output is 39
For input tensor([51, 39, 63,  1, 39]) output is 52
For input tensor([51, 39, 63,  1, 39, 52]) output is 57
For input tensor([51, 39, 63,  1, 39, 52, 57]) output is 61
For input tensor([51, 39, 63,  1, 39, 52, 57, 61]) output is 43
For input tensor([50]) output is 47
For input tensor([50, 47]) output is 44
For input tensor([50, 47, 44]) output is 44
For input tensor([50, 47, 44, 44]) output is 53
For input tensor([50, 47, 44, 44, 53]) output is 56
For input tensor([50, 47, 44, 44, 53, 56]) output is 42
For input tensor([50, 47, 44, 44, 53, 56, 42]) output is 1
For input tensor([50, 47, 44, 44, 53, 56, 42,  1]) output is 58
For input tensor([52]) output is 1
For input tensor([52,  1]) output is 58
For input tensor([52,  1, 58]) output is 46
For input tensor([52,  1, 58, 46]) output is 43
For input tensor([52,  1, 58, 46, 4

In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class Bigram(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,idx,targets):
        logits=self.token_embedding_table(idx) #B,T,C
        b,t,c=logits.shape
        
        pred=logits.view(b*t,c)
        actual_op=targets.view(b*t)
        loss=F.cross_entropy(pred,actual_op)
        return pred,loss
        
        

In [20]:
model=Bigram(vocab_size)

In [21]:
pred,loss=model(xb,yb)

In [22]:
pred.shape,loss

(torch.Size([32, 65]), tensor(4.7226, grad_fn=<NllLossBackward0>))

# Add generate function

In [23]:
class Bigram(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,idx,targets=None):
        logits=self.token_embedding_table(idx) #B,T,C
        if targets is None:
            loss=None
        else:
            b,t,c=logits.shape
        
            pred=logits.view(b*t,c)
            actual_op=targets.view(b*t)
            loss=F.cross_entropy(pred,actual_op)
        return logits,loss

    def generate(self,idx,new_token_len):
        
        for i in range(new_token_len):
            pred,loss=self(idx)
            op=pred[:,-1,:]
            prob=F.softmax(op,dim=-1)
            idx_n=torch.multinomial(prob,num_samples=1)
            idx=torch.cat([idx,idx_n],dim=1)
        return idx

In [24]:
idx=torch.zeros((1,1),dtype=torch.long)

In [25]:
m=Bigram(vocab_size)
decoder(m.generate(idx,new_token_len=100)[0].tolist())

"\nNm$nFkSxuQ.,OCJzRA:HHudd\n suQg'it& QYjW?lcxUSRl&auQTDqaqlQ$?T!:h V!;o3SQ$oBy,:xHfLLV,BrZFVLnl\ngxAEJe"

In [26]:
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [27]:
batch_size=32

for steps in range(100000):
    xb,yb=get_batch("train")
    
    logits,loss=m(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss.item())

2.483393907546997


In [28]:
decoder(m.generate(idx,new_token_len=100)[0].tolist())

'\nWensthe t r tinke o tifry, ou,\nMys t akse mimyoleetaththereusthe bl. w merigod d senot aul t ound? o'

# Version-1

In [29]:
B,T,C=4,8,2
x=torch.randn(B,T,C)

In [30]:
x[0,:]

tensor([[ 0.2049,  0.9165],
        [ 0.4723, -0.6560],
        [-1.6874, -1.0954],
        [-0.3887,  0.6235],
        [ 0.5212, -0.7762],
        [-0.5422,  0.1158],
        [ 0.0747,  0.2399],
        [ 0.7964,  0.0137]])

In [31]:
x.shape

torch.Size([4, 8, 2])

In [32]:
x_new=torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        v=x[b,:t+1]
        x_new[b,t]=torch.mean(v,dim=0)

In [33]:
x_new[0]

tensor([[ 0.2049,  0.9165],
        [ 0.3386,  0.1302],
        [-0.3368, -0.2783],
        [-0.3497, -0.0529],
        [-0.1755, -0.1975],
        [-0.2367, -0.1453],
        [-0.1922, -0.0903],
        [-0.0686, -0.0773]])

In [34]:
x[0]

tensor([[ 0.2049,  0.9165],
        [ 0.4723, -0.6560],
        [-1.6874, -1.0954],
        [-0.3887,  0.6235],
        [ 0.5212, -0.7762],
        [-0.5422,  0.1158],
        [ 0.0747,  0.2399],
        [ 0.7964,  0.0137]])

# Trick-1

In [35]:
m1=torch.tril(torch.ones(3,3))

In [36]:
m1

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [37]:
m1/m1.sum(dim=0)

tensor([[0.3333, 0.0000, 0.0000],
        [0.3333, 0.5000, 0.0000],
        [0.3333, 0.5000, 1.0000]])

In [38]:
a=torch.tril(torch.ones(3,3))

In [39]:
torch.sum(a,0,keepdim=True)

tensor([[3., 2., 1.]])

In [40]:
a/torch.sum(a,1,keepdim=True)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [41]:
a=torch.tril(torch.ones(T,T))
b=a/torch.sum(a,1,keepdim=True)

In [42]:
b

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [43]:
b@x #(B,T,T)*(B,T,C) -> (B,T,C)

tensor([[[ 0.2049,  0.9165],
         [ 0.3386,  0.1302],
         [-0.3368, -0.2783],
         [-0.3497, -0.0529],
         [-0.1755, -0.1975],
         [-0.2367, -0.1453],
         [-0.1922, -0.0903],
         [-0.0686, -0.0773]],

        [[-0.7840, -0.2144],
         [ 0.1645, -0.6343],
         [ 0.3665, -0.5104],
         [ 0.4518, -0.4599],
         [ 0.3205, -0.4364],
         [ 0.4775, -0.5468],
         [ 0.2650, -0.5147],
         [ 0.4679, -0.6071]],

        [[ 1.1486, -0.1009],
         [ 0.4030, -0.1396],
         [ 0.2041,  0.0557],
         [-0.3212,  0.1993],
         [-0.4843, -0.0561],
         [-0.5828, -0.3365],
         [-0.5377, -0.4169],
         [-0.4578, -0.2733]],

        [[ 0.2204, -0.5008],
         [-0.3666, -0.8449],
         [-0.1697, -0.7573],
         [-0.5061, -0.9290],
         [-0.6651, -0.7110],
         [-0.6606, -0.7147],
         [-0.4627, -0.7977],
         [-0.5397, -0.3950]]])

In [44]:
a=torch.tril(torch.ones(T,T))
y=torch.zeros(T,T)
y=y.masked_fill(a==0,float('-inf'))
y=F.softmax(y,dim=1)

In [45]:
y

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [46]:
B,T,C=4,8,32
x=torch.randn(B,T,C)

In [49]:
a=torch.tril(torch.ones(T,T))
w=torch.zeros(T,T)
w=w.masked_fill(a==0,float("-inf"))
w=F.softmax(w,dim=1)

In [50]:
w

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [51]:
out=w@x

In [58]:
head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)

k=key(x)
q=query(x)
v=value(x)
wei=q@k.transpose(-2,-1)

a=torch.tril(torch.ones(T,T))

wei=wei.masked_fill(a==0,float('-inf'))
wei=F.softmax(wei,dim=-1) 

In [59]:
out=wei@v

In [60]:
out[0]

tensor([[-0.8717,  1.1732, -0.0344,  0.5634,  0.6967,  0.2282, -0.5844, -1.5496,
          1.3764,  1.2214, -0.8700,  0.1974, -0.2306,  0.1520, -1.1335, -0.4122],
        [-0.0492,  0.3485, -0.5118,  0.3175,  0.0885,  0.4387, -0.2025, -0.5174,
          0.5704,  0.2039, -0.8271,  0.3728,  0.1499, -0.6336, -0.5252, -0.3835],
        [-0.4048,  0.7500, -0.2025,  0.4898,  0.3892,  0.3548, -0.3943, -1.0038,
          1.0164,  0.6590, -0.8114,  0.2392,  0.0136, -0.2414, -0.7869, -0.3630],
        [ 0.2487,  0.4448,  0.0811,  0.6281,  0.1360,  0.5344, -0.1691, -0.4104,
          0.7618,  0.0273, -0.3972, -0.0949,  0.3049, -0.3410, -0.2836, -0.1495],
        [ 0.4989,  0.2975,  0.1295,  0.5626,  0.0310,  0.5515, -0.0259, -0.1929,
          0.4701, -0.1331, -0.1485, -0.2442,  0.2752, -0.2548, -0.0766, -0.1057],
        [ 0.2894,  0.2507, -0.0908,  0.3814,  0.1138,  0.5175, -0.1435, -0.3582,
          0.5902, -0.0163, -0.2699,  0.0270,  0.3231, -0.3576, -0.2075, -0.1072],
        [ 0.2797,  0.2

In [57]:
out=y@x